# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [2]:
import getpass
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
from pyspark.sql.types import BooleanType
from pyspark.sql.window import Window
import math
import helpers
import pickle

%load_ext autoreload
%autoreload 2

## Initialize the `SparkSession`

In [3]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '6g')
conf.set('spark.executor.instances', '6')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=project-rbsteinm>

In [4]:
# init spark session
spark = SparkSession(sc)

## Loading the data

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe:

In [5]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

In [6]:
# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

In [7]:
metadata.show(5)

+----------------+----------+---------+---------+------+
|            name|station_ID|     long|      lat|height|
+----------------+----------+---------+---------+------+
|       Bucuresti|         2|26.074412| 44.44677|     0|
|          Calais|         3| 1.811446|50.901549|     0|
|      Canterbury|         4| 1.075329|51.284212|     0|
|          Exeter|         5|-3.543547|50.729172|     0|
|Fideris, Bahnhof|         7| 9.733756|46.922368|   744|
+----------------+----------+---------+---------+------+
only showing top 5 rows



We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [8]:
metadata.count()

25935

In [9]:
# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

In [10]:
# convert to pandas dataframe
pandas_df = metadata.toPandas()
# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

In [11]:
# pandas_df.distance_to_zh.max()

In [12]:
# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)
# create dict of stations from pandas dataframe
stations = pandas_df.set_index('station_ID').to_dict('index')

### Cleaning main dataset

In [13]:
# load full data
# raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
raw_df = spark.read.load('/datasets/project/istdaten/2018/01', format='csv', header='true', inferSchema='true', sep=';')

In [14]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])

In [15]:
# refactor dates
df = df.withColumn('date', functions.from_unixtime(functions.unix_timestamp('date', 'dd.MM.yyyy')))
df = df.withColumn('schedule_arrival', functions.from_unixtime(functions.unix_timestamp('schedule_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_arrival', functions.from_unixtime(functions.unix_timestamp('real_arrival', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('schedule_dep', functions.from_unixtime(functions.unix_timestamp('schedule_dep', 'dd.MM.yyyy HH:mm')))
df = df.withColumn('real_dep', functions.from_unixtime(functions.unix_timestamp('real_dep', 'dd.MM.yyyy HH:mm')))

In [16]:
# add a column containing the weekday (monday=1, sunday=6)
df = df.withColumn('weekday', helpers.get_weekday(df.date))

In [17]:
# keep only the rows with stops near zurich
df = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [18]:
# there is still 51'571'541 rows in zurich area
# df.count()

In [19]:
# keep only date after the 10th of december, because the schedule changed
df = df.where(df.date > '10.12.2017 00:00')

In [20]:
# discard the rows when there is no stop here
df2 = df.where(df.no_stop_here == 'false')

## Modeling the network

### From stops to trips

In [21]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', helpers.date_choice(df2.schedule_arrival, df2.schedule_dep))
#df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', helpers.stop_type(df2.schedule_dep, df2.schedule_arrival))

In [22]:
trips = df2.select(['trip_id', 'date', 'schedule_time', 'stop_id', 'stop_type', 'schedule_arrival', 'schedule_dep']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [23]:
# duplicate the dataframe, shift the copy of one row and append it to the original
# this way, we have for each row the current stop and the next stop
w = Window().partitionBy(functions.col('trip_id')).orderBy(functions.col('trip_id'))
trips2 = trips.select("*", functions.lag("trip_id").over(w).alias("next_tid"))
trips2 = trips2.select("*", functions.lag("schedule_time").over(w).alias("next_time"))
trips2 = trips2.select("*", functions.lag("stop_id").over(w).alias("next_sid"))
trips2 = trips2.select("*", functions.lag("stop_type").over(w).alias("next_type"))
trips2 = trips2.select("*", functions.lag("schedule_arrival").over(w).alias("next_sched_arr"))
trips2 = trips2.select("*", functions.lag("schedule_dep").over(w).alias("next_sched_dep"))

trips2 = trips2.where(trips2.next_time.isNotNull())

In [24]:
#trips2.where(trips2.stop_type=='first').count()
#trips2.where(trips2.stop_type=='last').count()
#trips2.where(trips2.stop_type=='mid').count()

In [25]:
# create a new column telling if the edge is valid or not
# (i.e. if the stop and next stop are really part of the same ride)
trips3 = trips2.withColumn('is_valid', helpers.edge_is_valid(trips2.trip_id, trips2.schedule_time, trips2.stop_id, trips2.stop_type, trips2.next_tid, trips2.next_time, trips2.next_sid, trips2.next_type))

In [26]:
# keep only valid edges
trips4 = trips3.filter(trips3.is_valid=='true')

In [27]:
# trips4.select('stop_id', 'next_sid').distinct().count()
# gives 6606

# trips3.select('stop_id', 'next_sid').distinct().count()
# gives 8557

### For each day of the week, model the network
Get the edges of the network and the departure/arrival times for each trip (edge=trip)
We assume the schedule repeat every week, and we generate one schedule per weekday.
Days off have the same schedules as sundays.

In [28]:
# creating a model for each day of the week
# this code needs to be run only once
typical_monday = '2018-01-15 00:00:00'
typical_tuesday = '2018-01-16 00:00:00'
typical_wednesday = '2018-01-17 00:00:00'
typical_thursday = '2018-01-18 00:00:00'
typical_friday = '2018-01-19 00:00:00'
typical_saturday = '2018-01-20 00:00:00'
typical_sunday = '2018-01-21 00:00:00'
typical_week = [typical_monday,typical_tuesday,typical_wednesday,typical_thursday,typical_friday,typical_saturday,typical_sunday]

In [29]:
regenerate_models = False
days_names = ['monday','tuesday','wednesday','thursday','friday','saturday','sunday']

# generate one network for each weekday and store them in pickles
if regenerate_models:
    for (date, day_name) in zip(typical_week, days_names):
        network = (helpers.model_network(trips4, date))
        with open('./data/'+day_name+'.pickle', 'wb') as handle:
            helpers.network_to_datetime(network) # works inplace
            pickle.dump(network, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(str(day_name) + ' done')

In [32]:
def compute_walking_network(stations, max_walk_dist=2.5):
    """
    computes the time you need to walk between any two nodes in the network
    in the form {A -> B -> time} and {B -> A -> time}
    the data structure contains None if the distance to walk is > max_walk_dist
    """
    walking_network = dict.fromkeys(stations.keys(), dict())
    # TODO instead of Nones just dont put B as a neigh of A if it's not reachable
    for i in stations.keys():
        for j in stations.keys():
            distance = helpers.distance(stations[i]['long'], stations[i]['lat'], stations[j]['long'], stations[j]['lat'])    
            time = compute_walking_time(distance) if (distance < max_walk_dist) else None # compute walking time            
            walking_network[i][j] = time
            walking_network[j][i] = time
    return walking_network
                
def compute_walking_time(distance, walking_speed=5.04):
    """distance in km, speed in km/h, returns time in datetime format"""
    return timedelta(hours=(distance / walking_speed))

In [33]:
# load the networks from the pickles
models = []
for day in days_names:
    with open('./data/'+ day +'.pickle', 'rb') as handle:
        network = pickle.load(handle)
    models.append(network)
    print(day + ' loaded')

# compute walking network
walking_network = compute_walking_network(stations)
print('walking network loaded')

monday loaded
tuesday loaded
wednesday loaded
thursday loaded
friday loaded
saturday loaded
sunday loaded
walking network loaded


In [34]:
def shortest_path(models, walking_network, stations, source, destination, departure_time):
    edges = models[departure_time.weekday()] # get the network for the correct day of the week
    Q = set(stations.keys()) # deep copy
    dist = dict.fromkeys(Q, datetime.max) # distances to the source (= arrival time at each node)
    prev = dict.fromkeys(Q, (None, None, None)) # previous node (and dep/arr times) in the shortest path
    dist[source] = departure_time
    
    while Q:
        #print(str(len(Q)) + ' remaining nodes ...')
        unvisited_dist = {key: dist[key] for key in Q} # distances of unvisited nodes
        u = min(unvisited_dist, key=unvisited_dist.get) # u <- vertex in Q with minimum dist[u]
        
        if dist[u] == datetime.max:
            raise Exception('Only nodes with infinity distance in the queue')
        
        Q.remove(u) #remove u from Q
        
        # if this is the destination node, we can terminate
        if u == destination:
            path = []
            while prev[u][0] != source:
                assert(prev[u][0] is not None), 'Could not find a path from ' + str(stations[source].name) + ' to ' + str(stations[destination].name)
                path.insert(0,prev[u])
                u = prev[u][0]
            path.insert(0,prev[u]) # push the source at the beginning of the path
            return path
        
        current_time = dist[u]
        neighbors = set(edges[u].keys()) if u in edges else set() # u's neighbors by vehicule
        walk_neighbors = set(walking_network[u].keys())
        for v in neighbors.union(walk_neighbors):
            (dep_time, arr_time) = get_next_correspondance(edges, current_time, walking_network, u, v) # take the first correspondance
            # there is no more correspondance for this edge
            if dep_time is None:
                continue
            dist_u_v = arr_time - dep_time
            waiting_time = dep_time - current_time
            arr_v = current_time + waiting_time + dist_u_v # determine at what time you'll arrive to v
            # a shorter path to v has been found
            if arr_v < dist[v]:
                dist[v] = arr_v
                prev[v] = (u, dep_time, arr_time)
            
    
    raise Exception('No path was found from source to destination')
        
        
def get_next_correspondance(edges, current_time, walking_network, source, dest):
    """
    returns departure/arrival times of the first ride departing after the current time
    assumes that the list current_time is sorted according to departure times!
    """
    # if the edge exists for the rides
    if source in edges and dest in edges[source]:
        times = edges[source][dest]
        # index of the fist ride departing after the current time
        index = np.searchsorted([x[0] for x in times], current_time)
        (dep,arr) = times[index] if index < len(times) else (None,None) # None if there is no more ride at this time
    else:
        (dep,arr) = (None,None) # None if there is no edge by vehicule
        
    walk_time = walking_network[source][dest]
    (dep_walk,arr_walk) = (current_time,current_time+walk_time) if walk_time is not None else (None,None)
    if dep is None and dep_walk is None:
        return (None,None) # if you can neither walk nor take a transport
    elif dep is None:
        return (dep_walk,arr_walk) # if you can only walk
    elif dep_walk is None:
        return (dep,arr) # if you can only take a transport
    else:
        return (dep,arr) if (arr<arr_walk) else (dep_walk,arr_walk) # if you can walk or ride, take the fastest

In [35]:
shortest_path(models, walking_network, stations, 8591353, 8591421, datetime(2018, 1, 15, 14))

Exception: Only nodes with infinity distance in the queue

In [ ]:
import random
shortest_paths = []
for i in range(10):
    source = random.choice(list(stations.keys()))
    dest = random.choice(list(stations.keys()))
    shortest_paths.append(shortest_path(models, walking_network, stations, source, dest, datetime(2018, 1, 15, 14)))
    

In [ ]:
shortest_paths[4]

In [ ]:
path = shortest_path(models, walking_network, stations, 8576152, 8573715, datetime(2018, 1, 15, 14))

In [ ]:
path

In [ ]:
models[0][8503610][8573711]